Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [80]:
import numpy as np

In [100]:
rate = pd.read_csv ('ratings.csv') # данные с рейтингами
rate.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [108]:
user = (
    rate.groupby(rate.userId)
    .count()[['timestamp']]
    .sort_values(by=['timestamp'], ascending =False)
)           # группировка кол-ва оценок по клиенту
user100 = user.loc[user.timestamp >100].reset_index() # Фильтрация клиентов у которых более 100 оценок
user100.columns = ['userId', 'count_timestamp']
user100

,userId,count_timestamp
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346
...,...,...
240,309,103
241,40,103
242,412,102
243,32,102


In [116]:
times = ((
    rate.groupby(rate.userId).max()[['timestamp']]-rate.groupby(rate.userId).min()[['timestamp']])
    .reset_index()
)
times.columns = ['userId', 'time_life']
times  #среднее время жизни пользователей

,userId,time_life
0,1,739163
1,2,505
2,3,970
3,4,62496114
4,5,590
...,...,...
605,606,197231731
606,607,34768995
607,608,72402187
608,609,278


In [117]:
users = user100.merge(times, on='userId',how='left')
users.sort_values(by=['count_timestamp'], ascending =False)
# решение

,userId,count_timestamp,time_life
0,414,2698,566541856
1,599,2478,21102011
2,474,2108,263684276
3,448,1864,480496843
4,274,1346,147238067
...,...,...,...
240,309,103,85771335
241,40,103,1588
242,412,102,449467
243,32,102,1269


Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [19]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)


In [20]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)


In [21]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)


In [18]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)


In [32]:
stat_total = (
    rzd.merge(auto, on='client_id', how='outer')
    .merge(air, on='client_id', how='outer')
) 
stat_total    #Таблица1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [31]:
clients_info = (
    client_base.merge(rzd, on='client_id',how='left')
    .merge(auto, on='client_id', how='left')
    .merge(air, on='client_id', how='left')
)
clients_info   #Таблица2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1)У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2)Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3)Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Если последовательность содержит время фиксации координат.Привязываем координаты через условие, что время их фиксации попадает в диапазон : время активности +/- N минут
2. Если последовательность не содержит время фиксации координат. Для офлайн магазинов можно создать датафрейм с их диапазоном координат и анализировать попадание координат клиента в диапазон координат магазина
3. По координатам  в привязке к картам и времени можно определить например местожительство (ночные часы), работы (дневные часы) и мест периодического пребывания
